In [1]:
import pandas as pd
import requests
import datetime
from geopy.geocoders import OpenMapQuest

In [2]:
sites_location=pd.read_csv('../data/sites_location.csv', delimiter =';')

In [3]:
sites_location.head()

,site_id,City,UTC_offset
0,0,Jacksonville,-4
1,1,London,0
2,2,Phoenix,-7
3,3,Philadelphia,-4
4,4,San Francisco,-7


In [4]:
def location(my_address):
    geolocator = OpenMapQuest(api_key='LfGsouoBmQGZZAGpkFZzXGyzo27zOFK2')
    location = geolocator.geocode(my_address, timeout=10)
    return (location.latitude, location.longitude)

sites_location['lat_lon']=sites_location['City'].apply(location)

In [5]:
params={
'parameters':'CDD0,CDD10,HDD18_3,T2M,ALLSKY_SFC_LW_DWN,ALLSKY_TOA_SW_DWN,ALLSKY_SFC_SW_DWN',
'start_date':'20160101',
'end_date':'20190101',
'user_community':'SSE'
       }

In [77]:
def download_data(lat, lon, parameters, start_date, end_date, user_community):
    
    api_url='https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?request=execute&identifier=SinglePoint&parameters={url_parameters}&startDate={start_date}&endDate={end_date}&userCommunity={user_community}&tempAverage=DAILY&outputList=JSON&lat={lat}&lon={lon}&user=anonymous'
    
    request_url = api_url.format(url_parameters=parameters, start_date=start_date, end_date=end_date, user_community=user_community, lat=lat, lon=lon)
    response = requests.get(request_url)
    
    if response.status_code == 200:
        df=pd.DataFrame(response.json()['features'][0]['properties']['parameter'])
        df['date']=pd.date_range(start_date, end_date)
        
        
        return df
    
    else:
        None


def extract(df):
    result=pd.DataFrame()
    
    for site_id in df['site_id']:
        lat, lon=df[df['site_id']==site_id]['lat_lon'][site_id]
        
        data = download_data(lat, lon, **params)
        print('Data downloaded for site_id #', site_id)
        
        data['site_id']=site_id
        
        result=result.append(data, ignore_index=True)
        del data
    return result

In [70]:
nasa=extract(sites_location)

Data downloaded for site_id # 0
Data downloaded for site_id # 1
Data downloaded for site_id # 2
Data downloaded for site_id # 3
Data downloaded for site_id # 4
Data downloaded for site_id # 5
Data downloaded for site_id # 6
Data downloaded for site_id # 7
Data downloaded for site_id # 8
Data downloaded for site_id # 9
Data downloaded for site_id # 10
Data downloaded for site_id # 11
Data downloaded for site_id # 12
Data downloaded for site_id # 13
Data downloaded for site_id # 14
Data downloaded for site_id # 15


In [73]:
nasa.sample(10)

,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN,ALLSKY_TOA_SW_DWN,CDD0,CDD10,CLRSKY_SFC_SW_DWN,HDD18_3,T2M,Date,site_id
2951,6.18,4.17,5.77,13.31,3.31,4.28,4.99,12.11,2018-01-27,2
13637,7.70,3.54,8.73,8.13,0.00,-999.00,10.17,8.22,2017-04-18,12
5689,8.66,5.57,10.71,19.10,9.10,-999.00,0.00,19.49,2016-07-23,5
11499,6.73,8.79,11.55,20.64,10.64,8.95,0.00,21.70,2017-06-13,10
16902,5.26,5.99,8.20,1.10,0.00,5.99,17.20,0.22,2017-03-23,15
11883,8.01,8.38,11.50,31.49,21.49,8.56,0.00,31.16,2018-07-02,10
7474,8.57,3.54,11.55,15.95,5.95,-999.00,2.35,15.71,2018-06-11,6
14953,6.25,2.20,3.49,0.00,0.00,2.21,18.36,0.17,2017-11-23,13
8212,9.19,5.42,11.60,21.84,11.84,-999.00,0.00,21.91,2017-06-17,7
17241,7.54,1.86,6.31,9.29,0.00,-999.00,9.01,10.08,2018-02-25,15


In [76]:
nasa.to_csv('../data/nasa_weather.csv', index=False)